In [1]:
!pip install -q openai

In [1]:
import openai
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY'
openai.api_key = os.getenv('OPENAI_API_KEY')

# Document Data

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("unicorns.csv")

In [5]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Updated at                  1199 non-null   object 
 1   Company                     1199 non-null   object 
 2   Crunchbase Url              1199 non-null   object 
 3   Last Valuation (Billion $)  1199 non-null   float64
 4   Date Joined                 1199 non-null   object 
 5   Year Joined                 1199 non-null   int64  
 6   City                        1181 non-null   object 
 7   Country                     1199 non-null   object 
 8   Industry                    1199 non-null   object 
 9   Investors                   1199 non-null   object 
 10  Company Website             0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 103.2+ KB


lets create a new columns ("summary columns")

In [9]:
import ast

In [7]:
def summary(company, crunchbase_url, city, country, industry, investor_list):
  investors = "The investors in the company are "
  for investor in ast.literal_eval(investor_list):
      investors += f"{investor},"

  text = f"{company} has headquarters in {city} in {country} and is in the field of {industry}. {investors}. More info at {crunchbase_url}"

  return text


In [10]:
df['summary'] = df.apply(lambda df: summary(df['Company'],df['Crunchbase Url'],df['City'],df['Country'],df['Industry'],df['Investors']),axis=1)

In [11]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...


In [15]:
import textwrap
text=df['summary'][1]
wrapped_text = textwrap.fill(text, width=100)
print(wrapped_text)

Fever Labs has headquarters in New York in United States and is in the field of Internet software &
services. The investors in the company are Accel,14W,GS Growth,. More info at
https://www.cbinsights.com/company/fever-labs


# Token Count and Pricing

In [16]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00


In [17]:
import tiktoken

In [18]:
def num_tokens_from_string(string, encoding_name):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [19]:
num_tokens_from_string(df['summary'][0], "cl100k_base")

52

In [20]:
len(df['summary'][0].split())

32

In [21]:
df["token_count"] = df['summary'].apply(lambda text: num_tokens_from_string(text, "cl100k_base"))

In [22]:
int(df["token_count"].sum())

63363

In [24]:
# $0.0001 per 1k token USD
# total price for the text_embedding
float(df["token_count"].sum()*0.0001 /1000)

0.0063363000000000004

In [25]:
def get_embedding(text):
    response = openai.embeddings.create(
        input=text,
        model='text-embedding-ada-002'
    )
    return response.data[0].embedding

In [26]:
vector = get_embedding("Esusu has headquarters in New York in United States and is in the field of Fintech. SoftBank Group SoftBank Group,. Nore info at https://www.cbinsights.com/company/esusu")

In [27]:
len(vector)

1536

In [28]:
df['embedding'] = df['summary'].apply(get_embedding)

In [29]:
df

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.00,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,52,"[0.011081160977482796, -0.018728159368038177, ..."
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.00,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,53,"[0.008602230809628963, 0.011036576703190804, -..."
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.00,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,51,"[0.005523009225726128, -0.04294928163290024, 0..."
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.00,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,56,"[-6.426063919207081e-05, -0.02552231401205063,..."
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.00,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,53,"[0.011503264307975769, -0.01382159348577261, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,"10/31/2022, 2:32:42 AM",Fanatics,https://www.cbinsights.com/company/fanatics,27.00,6/6/2012,2012,Jacksonville,United States,E-commerce & direct-to-consumer,"[""SoftBank Group"",""Andreessen Horowitz"",""Temas...",NaN,Fanatics has headquarters in Jacksonville in U...,57,"[-0.0017611222574487329, -0.012050825171172619..."
1195,"10/31/2022, 2:32:38 AM",SpaceX,https://www.cbinsights.com/company/space-explo...,127.00,12/1/2012,2012,Hawthorne,United States,Other,"[""Founders Fund"",""Draper Fisher Jurvetson"",""Ro...",NaN,SpaceX has headquarters in Hawthorne in United...,59,"[0.02533082477748394, -0.014380521140992641, -..."
1196,"10/31/2022, 2:33:10 AM",Vice Media,https://www.cbinsights.com/company/vice-media,5.70,4/2/2011,2011,Brooklyn,United States,Internet software & services,"[""Technology Crossover Ventures"",""A&E Televisi...",NaN,Vice Media has headquarters in Brooklyn in Uni...,50,"[0.007336130365729332, -0.03166935220360756, 0..."
1197,"10/31/2022, 2:33:05 AM",Klarna,https://www.cbinsights.com/company/klarna,6.70,12/12/2011,2011,Stockholm,Sweden,Fintech,"[""Institutional Venture Partners"",""Sequoia Cap...",NaN,Klarna has headquarters in Stockholm in Sweden...,51,"[0.026691757142543793, -0.025004049763083458, ..."


In [30]:
df.to_csv('my_embedding.csv')

# Document Similarity and Context Injection

In [31]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,52,"[0.011081160977482796, -0.018728159368038177, ..."
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,53,"[0.008602230809628963, 0.011036576703190804, -..."
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,51,"[0.005523009225726128, -0.04294928163290024, 0..."
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,56,"[-6.426063919207081e-05, -0.02552231401205063,..."
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,53,"[0.011503264307975769, -0.01382159348577261, 0..."


In [32]:
prompt = "What does the company Minio do and where is the HQ?"

In [33]:
prompt_embedding = get_embedding(prompt)

In [34]:
prompt_embedding

[-0.0021560178138315678,
 -0.027890315279364586,
 0.005628341343253851,
 -0.03463594615459442,
 -0.04399323835968971,
 0.03240136802196503,
 -0.016857091337442398,
 -0.007443935144692659,
 0.014468886889517307,
 -0.008191121742129326,
 0.028853977099061012,
 0.008624071255326271,
 0.0027844926808029413,
 -0.030809232965111732,
 0.006462816148996353,
 0.007876884192228317,
 -0.0012621869100257754,
 -0.016396209597587585,
 0.027429433539509773,
 -0.00024746195413172245,
 -0.009671528823673725,
 0.015977226197719574,
 -0.020935194566845894,
 -0.011480139568448067,
 0.014161632396280766,
 -0.015362718142569065,
 0.015991192311048508,
 -0.005802917294204235,
 0.004046679474413395,
 -0.012688208371400833,
 0.01607498899102211,
 -0.010900546796619892,
 -0.004004781134426594,
 0.00015089941734913737,
 0.0017248141812160611,
 0.007786104921251535,
 0.028448959812521935,
 0.004710069391876459,
 0.004273628816008568,
 -0.014734243042767048,
 -0.003627696307376027,
 -0.0043364763259887695,
 0.0070

In [35]:
import numpy as np

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
def vector_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]

In [37]:
df['prompt_similarity'] = df['embedding'].apply(lambda vector:vector_similarity(vector, prompt_embedding))

In [38]:
df['prompt_similarity']

,prompt_similarity
0,0.734976
1,0.736571
2,0.885993
3,0.758952
4,0.748027
...,...
1194,0.738153
1195,0.752419
1196,0.758349
1197,0.756377


In [39]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding,prompt_similarity
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,52,"[0.011081160977482796, -0.018728159368038177, ...",0.734976
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,53,"[0.008602230809628963, 0.011036576703190804, -...",0.736571
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,51,"[0.005523009225726128, -0.04294928163290024, 0...",0.885993
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,56,"[-6.426063919207081e-05, -0.02552231401205063,...",0.758952
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,53,"[0.011503264307975769, -0.01382159348577261, 0...",0.748027


In [40]:
df.nlargest(1, 'prompt_similarity')

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,token_count,embedding,prompt_similarity
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,51,"[0.005523009225726128, -0.04294928163290024, 0...",0.885993


Now we can easily grab the summary for the most similar embedding, then insert that as context to our GPT request!

In [41]:
df.nlargest(1, 'prompt_similarity').iloc[0]['summary']

'Minio has headquarters in Palo Alto in United States and is in the field of Data management & analytics. The investors in the company are General Catalyst,Nexus Venture Partners,Dell Technologies Capital,. More info at https://www.cbinsights.com/company/minio'

## Question Answering with Embeddings

In [42]:
Context = df.nlargest(1, 'prompt_similarity').iloc[0]['summary']

In [43]:
prompt = f''' Only answer the question below if you have 100% certainty of the facts.
Context : {Context}
Q: What does the start-up company Pentera do and who invested in it?
A:
'''

In [44]:
print(prompt)

 Only answer the question below if you have 100% certainty of the facts.
Context : Minio has headquarters in Palo Alto in United States and is in the field of Data management & analytics. The investors in the company are General Catalyst,Nexus Venture Partners,Dell Technologies Capital,. More info at https://www.cbinsights.com/company/minio
Q: What does the start-up company Pentera do and who invested in it?
A:



In [45]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=0,
    max_tokens=512
)

In [46]:
response

ChatCompletion(id='chatcmpl-BP4ARcsoaNV0aMgVDQnZ9h38MdjYE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I cannot answer the question about Pentera as the information provided is about Minio.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745313919, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=97, total_tokens=116, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [47]:
response.choices[0].message.content

'I cannot answer the question about Pentera as the information provided is about Minio.'